In [ ]:
import statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
data = pd.read_csv('Truck_sales.csv', usecols=[1])
data_array = data.to_numpy().flatten()
print(data_array)

In [ ]:
graph_data = data_array.copy()
plt.plot(data_array)
plt.xticks([0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144])
plt.show()

### Make data non-seasonal
Remove Seasonality:
$z_t = y_t - y_{t-12}$

In [ ]:
nonseasonal_data = data.diff(-12)
nonseasonal_data = nonseasonal_data.dropna()
nonseasonaldata_array = nonseasonal_data.to_numpy().flatten()
print(nonseasonaldata_array)

In [ ]:
graph_nonseasonaldata = nonseasonaldata_array.copy()
plt.plot(nonseasonaldata_array)
plt.xticks([0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144])
plt.show()

### Check for white noise
i) $\mu = 0$

ii) $\sigma^2$ is constant

iii) correlation between lags is 0

In [ ]:
# Augmented Dickey-Fuller Test for non-stationarity

# result = adfuller(nonseasonal_data)
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])

In [ ]:
# Check mean
mean = np.average(nonseasonal_data)
print("Mean: %f" % mean)

print('Not white noise.')

### $\sigma$ and $\mu$ is constant by inspection

### Run ACF and PACF

In [ ]:
acf_plot = plot_acf(nonseasonal_data, lags=131)

In [ ]:
pacf_plot = plot_pacf(nonseasonal_data)

### Fit ARIMA(3,3,0) model to data

In [ ]:
model = ARIMA(nonseasonal_data, order=(2,0,0))
results = model.fit()
print(results.summary())

### Predict future prices from ARIMA(3,3,0)

In [ ]:
predictions = results.predict(start=132, end=144, dynamic=False, signal_only=False, information_set='predicted')

In [ ]:
predictions = predictions.to_numpy()
print(predictions)

In [ ]:
# Unnormalize our predictions

unnorm_predictions = predictions
for i in range(len(predictions)-1):
    unnorm_predictions[i] += nonseasonaldata_array[120+i]


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(graph_data, label='Data', color='blue')
plt.plot(range(132, 144), unnorm_predictions[:12], label='Predictions', color='red')
plt.xticks([12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144])
plt.legend()
plt.show()